In [ ]:
import ipynb
import data
import pandas as pd
import pandas.tseries.offsets as offset


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def date_control(x):
    dates = x.dropna().index.to_series()

    start = dates.diff() !=1

    end = dates.shift(-1)-dates != 1

    start = dates[start].values
    end = dates[end].values

    return list(zip(start,end))

In [ ]:
def date_control_quarter(x):
    dates = x.dropna().index.to_series()

    start = dates.diff() < offset.Day(95)
    end = dates.shift(-1)-dates < offset.Day(95)

    start = dates[~start].values
    end = dates[~end].values
    
    start = pd.PeriodIndex(start,freq='Q')
    end = pd.PeriodIndex(end, freq='Q')

    return list(zip(start,end))

In [ ]:
def count(df, name=None,freq='a'):
    df['date'] = pd.Series(df.index)
    if freq == 'q':
        df = df.drop('indicator',axis=1).set_index('date')
        df_year = df.apply(date_control_quarter)
    elif freq == 'a':
        df.date = df.date.dt.year
        df = df.drop('indicator',axis=1).set_index('date')
        df_year = df.apply(date_control)
    else:
        pass
    if isinstance(df_year,pd.DataFrame):
        df_year =df_year.loc[0]
    df_result = pd.concat([df_year,df.count()],axis=1)
    df_result.columns = ['start-end','total']
    if name is not None:
        df_result['name'] = name
    return df_result

In [ ]:
for name in data.indicator('wb').code:
    count(data.read_wb(name)).to_csv('date_control/wb/{}.csv'.format(name))

In [ ]:
for name in data.indicator('imf').code:
    count(data.read_imf(name,'a'),name,'a').to_csv('date_control/imf/annually/{}.csv'.format(name))
    count(data.read_imf(name,'q'),name,'q').to_csv('date_control/imf/quarterly/{}.csv'.format(name))

In [ ]:
df = data.read_wb('SP.POP.GROW')

In [ ]:
x=df.index.year.to_series()

In [ ]:
x

In [ ]:
df.apply(date_control_quarter).to_csv('asd.csv')